<span class='note'>*Make me look good.* Click on the cell below and press <kbd>Ctrl</kbd>-<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open('css/custom.css', 'r').read()
    return HTML(styles)
css_styling()

<h5 class='prehead'>SA367 &middot; Mathematical Models for Decision Making &middot; Spring 2018 &middot; Uhan</h5>

<h1 class='lesson_title'>Quiz 8 &mdash; 24 April 2017</h1>

## Instructions

This quiz is due **at the end of class on Thursday 26 April**.

Type your answers to all of the problems **directly in this Jupyter notebook**, and submit this notebook (just the `ipynb` file) using the [SA367 submission form](https://script.google.com/a/usna.edu/macros/s/AKfycbztmCZWUZwdekL5yk-VJvfrpJrC_cOgP9onlTFp7bJSJDQgkh98/exec).

**You may work with other midshipmen, but each midshipman must submit his or her own quiz.** You may use any of your own course materials (e.g., notes, homework) as well as any materials linked from the course website.

## Problem 1

Place your code in the cell below.

In [ ]:
# Setup code
from stochasticdp import StochasticDP

# Total number of possessions
# Drive Averages: 2 * (#Dr) / (G * (# of teams))
T = 23

# 1-pt. conversion success probabilities
# Kicking and Punting: XP%
pA = 0.936
pB = 0.936

# 2-pt. conversion success probabilities
# Scoring Offense: 2PM / 2PA
qA = 0.486
qB = 0.486

# 1-pt. vs 2-pt attempts
# 1-pt.: Scoring Offense: XPA / (XPA + 2PA)
b1 = 0.919
b2 = 1 - b1

# Possession outcome probabilities - Team A
# TD: (Scoring Offense: ATD) / (Drive Averages: #Dr)
# FG: (Scoring Offense: FGM) / (Drive Averages: #Dr)
tA = 0.223
gA = 0.145
zA = 1 - tA - gA

# Possession outcome probabilities - Team B
tB = 0.223
gB = 0.145
zB = 1 - tB - gB

# Probability that Team A wins in OT
r = 0.5

# Number of stages
number_of_stages = T + 1

# Maximum lead for Team A
max_d = 20

# List of states
states = []
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-max_d, max_d + 1):
            states.append((n, k, d))
            
# List of decisions
decisions = [1, 2, 'none']

# Initialize stochastic dynamic program - we want to maximize, so minimize = False
dp = StochasticDP(number_of_stages, states, decisions, minimize=False)

# Transition probabilities from (A, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 6, max(d - 6, -20)), tB, 0)
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 3, max(d - 3, -20)), gB, 0)
        dp.add_transition(t, ('A', 0, d), 'none', ('B', 0, d), zB, 0)
        
# Transition probabilities from (A, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 6, max(d - 6, -20)), tB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 3, max(d - 3, -20)), gB, 0)
        dp.add_transition(t, ('A', 3, d), 'none', ('B', 0, d), zB, 0)  

# Transition probabilities from (A, 6, d) up to stage T - 2
# Decision: 1-point conversion
for t in range(T - 1):
    for d in range(-20, 21):

        if d - 5 <= -20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, -20), (1 - pA) * tB + pA * tB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, max(d - 6, -20)), (1 - pA) * tB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 6, max(d - 5, -20)), pA * tB, 0)
        
        if d - 2 <= -20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, -20), (1 - pA) * gB + pA * gB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, max(d - 3, -20)), (1 - pA) * gB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 3, max(d - 2, -20)), pA * gB, 0)
        
        if d >= 20:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, 20), (1 - pA) * zB + pA * zB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, min(d, 20)), (1 - pA) * zB, 0)
            dp.add_transition(t, ('A', 6, d), 1, ('B', 0, min(d + 1, 20)), pA * zB, 0)
            
# Transition probabilities from (A, 6, d) up to stage T - 2
# Decision: 2-point conversion
for t in range(T - 1):
    for d in range(-20, 21):

        if d - 4 <= -20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, -20), (1 - qA) * tB + qA * tB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, max(d - 6, -20)), (1 - qA) * tB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 6, max(d - 4, -20)), qA * tB, 0)
        
        if d - 1 <= -20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, -20), (1 - qA) * gB + qA * gB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, max(d - 3, -20)), (1 - qA) * gB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 3, max(d - 1, -20)), qA * gB, 0)
            
        if d >= 20:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, 20),  (1 - qA) * zB + qA * zB, 0)
        else:
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, min(d, 20)), (1 - qA) * zB, 0)
            dp.add_transition(t, ('A', 6, d), 2, ('B', 0, min(d + 2, 20)), qA * zB, 0)

# Transition probabilities from (B, 0, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 6, min(d + 6, 20)), tA, 0)
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 3, min(d + 3, 20)), gA, 0)
        dp.add_transition(t, ('B', 0, d), 'none', ('A', 0, d), zA, 0)
        
# Transition probabilities from (B, 3, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 6, min(d + 6, 20)), tA, 0)
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 3, min(d + 3, 20)), gA, 0)
        dp.add_transition(t, ('B', 3, d), 'none', ('A', 0, d), zA, 0)        

# Transition probabilities from (B, 6, d) up to stage T - 2
for t in range(T - 1):
    for d in range(-20, 21):
        if d + 4 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, 20), (1 - b1*pB - b2*qB) * tA + b1*pB*tA + b2*qB*tA, 0)
        elif d + 5 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, 20), (1 - b1*pB - b2*qB) * tA + b1*pB*tA, 0)    
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 4, 20)), b2*qB*tA, 0)
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 6, 20)), (1 - b1*pB - b2*qB) * tA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 5, 20)), b1*pB*tA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 6, min(d + 4, 20)), b2*qB*tA, 0)

        if d + 1 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, 20), (1 - b1*pB - b2*qB) * gA + b1*pB*gA + b2*qB*gA, 0)
        elif d + 2 >= 20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, 20), (1 - b1*pB - b2*qB) * gA + b1*pB*gA, 0)    
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 1, 20)), b2*qB*gA, 0)
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 3, 20)), (1 - b1*pB - b2*qB) * gA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 2, 20)), b1*pB*gA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 3, min(d + 1, 20)), b2*qB*gA, 0)
            
        if d <= -20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, -20), (1 - b1*pB - b2*qB) * zA + b1*pB*zA + b2*qB*zA, 0)
        elif d - 1 <= -20:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, d), (1 - b1*pB - b2*qB) * zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, -20), b1*pB*zA + b2*qB*zA, 0)    
        else:
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, d), (1 - b1*pB - b2*qB) * zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, max(d - 1, -20)), b1*pB*zA, 0)
            dp.add_transition(t, ('B', 6, d), 'none', ('A', 0, max(d - 2, - 20)), b2*qB*zA, 0)
        
# Transition probabilities from (A, 0, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('A', 0, d), 'none', ('A', 0, d), 1, 0)
    
# Transition probabilities from (A, 3, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('A', 3, d), 'none', ('A', 3, d), 1, 0)
    
# Transition probabilities from (A, 6, d) in stage T - 1
for d in range(-20, 21):
    # 1-point conversion
    if d >= 20:
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, 20), (1 - pA) + pA, 0)
    else:
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, d), 1 - pA, 0)
        dp.add_transition(T - 1, ('A', 6, d), 1, ('A', 6, min(d + 1, 20)), pA, 0)

    # 2-point conversion
    if d >= 20:
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, 20), (1 - qA) + qA, 0)
    else:
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, d), 1 - qA, 0)
        dp.add_transition(T - 1, ('A', 6, d), 2, ('A', 6, min(d + 2, 20)), qA, 0)

# Transition probabilities from (B, 0, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('B', 0, d), 'none', ('B', 0, d), 1, 0)
    
# Transition probabilities from (B, 3, d) in stage T - 1
for d in range(-20, 21):
    dp.add_transition(T - 1, ('B', 3, d), 'none', ('B', 3, d), 1, 0)
    
# Transition probabilities from (B, 6, d) in stage T - 1
for d in range(-20, 21):
    if d <= -20:
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, -20), 1, 0)
    elif d <= -19:
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, -20), b1*pB + b2*qB, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, d), 1 - b1*pB - b2*qB, 0)
    else:        
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, max(d - 2, -20)), b2*qB, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, max(d - 1, -20)), b1*pB, 0)
        dp.add_transition(T - 1, ('B', 6, d), 'none', ('B', 6, d), 1 - b1*pB - b2*qB, 0)

# Boundary conditions
for n in ['A', 'B']:
    for k in [0, 3, 6]:
        for d in range(-20, 21):
            if d > 0:
                dp.add_boundary(state=(n, k, d), value=1)
            elif d == 0:
                dp.add_boundary(state=(n, k, d), value=r)
            else:
                dp.add_boundary(state=(n, k, d), value=0)
                
# Solve the stochastic dynamic program
value, policy = dp.solve()

## Problem 2

Place your code in the cell below.

In [ ]:
value[0, ('A', 3, 3)]

## Problem 3

Place any code that helps you answer the problem in the cell below.

In [ ]:
d = -4
for t in range(number_of_stages - 1):
    print("Points ahead: {1}   Possession: {0}    Go for: {2}   Pr(win): {3}"
          .format(t, d, policy[t, ('A', 6, d)], value[t, ('A', 6, d)]))

<!-- _Write your response here. Double-click to edit._ -->

At the beginning of the game, Team A in this situation should go for 2. Towards to the end of the game, starting at possession 8, Team A should go for 1 in this situation.

On the other hand, assuming 2 average NFL teams in the 2014 season, Team A should always go for 1. 

The differences between the optimal strategies in 2014 and 2016 make sense in light of the rule change on 1-point conversions in 2015. In 2014, a successful 1-point conversion was nearly guaranteed. On the other hand, in 2016, a 1-point conversion had a lower success rate due to the rule change, making going for 2 in 2016 a more palatable option earlier in the game.

## Problem 4

<!-- _Write your response here. Double-click to edit._ -->

There are several aspects of a real-world football game that could be added to our formulation. Two examples:

- __Actual time remaining in game.__ Including the actual time remaining would allow our model to consider having more or less time in the final few possessions, which could affect the points-after-touchdown decision. In general, it would also allow for a more realistic representation of the length of a football game.

- __Modeling Team B's points-after-touchdown decision.__ Instead of assuming that Team B goes for 1 some fraction of the time ($b_1$), and goes for 2 the rest of the time ($b_2 = 1 - b_1$), we could explicitly model Team B's points-after-touchdown decision, assuming that Team B acts optimally in order to _minimize_ the probability of Team A winning. This would require a different type of value-to-recursion than the one we discussed in class, one that alternates between maximizing (on Team A's possessions) and minimizing (on Team B's possessions).

## When you're finished

* Select **Cell &#8594; Run All**  to run all the cells from top to bottom. Make sure your notebook runs as you expect before submitting it.